In [1]:
%load_ext watermark

In [2]:
%watermark -a Schmelling,Nicolas -u -d -v -p numpy,pandas,scipy,biopython

Schmelling,Nicolas 
Last updated: 29/09/2015 

CPython 2.7.9
IPython 3.0.0

numpy 1.9.2
pandas 0.15.2
scipy 0.15.1
biopython 1.65


---
Any comments and suggestions or questions?     
Please feel free to contact me via [twitter](https://twitter.com/bio_mediocre) or [email](mailto:schmelli@msu.edu).

---

<a id='Content'></a>
-----

#Content#
---

| [Biopython BLAST](#BLAST) | [Download Information from GenBank](#genbank) | [Command line BLAST](#BLAST back) | [Filtering of the reciprocal BLAST](#filter) | [Merging and final filtering](#CSV) |

---

In [3]:
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
from Bio import SeqIO
from Bio import Phylo
from Bio import AlignIO
from Bio import Entrez
from urllib2 import HTTPError
import glob
import time
import csv
import pandas as pd

###BLAST###
<a id='BLAST'></a>
-----
---

Before we get started with our BLAST run, we need a fasta file with our sequences. So either you copy/paste your sequences of interest in a new fasta file or you download a file that contains all your sequences. In my case I needed to get all sequences and create a new fasta file.      

Your fasta file should look something like this:          

`> fasta sequence header`     
`sequence e.g. MASJINDHASDINEIHGNISD`      
`> next sequence header`      
`next sequence e.g. MINASIDJGHIENGISJDA`     

In [4]:
# I used the Biopython package for all BLAST related analysis and coding. For more information check www.biopython.org.
# First we read our FASTA file sequence by sequence. The SeqIO.parse function needs only a file name, but to make sure it reads
# the file correct, we specify the file format.
for seq_record in SeqIO.parse('clock_proteins_Arabidopsis.faa', format='fasta'):
    
    # Next we send our sequence to NCBI server and use the BLAST online version. 
    # NCBIWWW takes a program argument: In our case blastp, b/c we blast protein against protein. 
    # a database argument: In our case nr, b/c we want to blast against all non-redundant protein sequences.
    # a format argument: Just to make sure BLAST reads our sequences correctly.
    # and various additional arguments (optinal): expect = e-value (cut off value, for less similarity), 
                                                # hitlist_size = no. of maximum hits
                                                # matrix_name and word_size are the default values.
    result_handle = NCBIWWW.qblast('blastp', 'nr', seq_record.format('fasta'), expect=0.00001,
                                   hitlist_size=10000, matrix_name='BLOSUM62', word_size=3)
    
    # We save the BLAST result for every sequence in a new XML file, for later parsing.
    # First we need to open a new file with a descriptive name (sequence name seems like a good idea)
    # Then we write the results in the file and don't forget to close it afterwards.
    # In the end we also close the BLAST result file, before we go on to the next sequence.
    save_file = open(seq_record.id + '_BLAST.xml', 'w')
    save_file.write(result_handle.read())
    save_file.close()
    result_handle.close()   
# This script can run a while, depending on the no. of sequences in your file and your hitlist_size value.

[Back to Content](#Content)

<a id='genbank'></a>
-----

---
###Download Information from GenBank###
---

Before we blast our hits against the Arabidopsis genome, we extract information out these hits from GenBank and write these information into a CSV file. Afterwards we extract the amino acid sequences from each hit and write a FASTA file for the reciprocal BLAST.

---

In [5]:
for xmlfile in glob.glob('*.xml'):
    
    # Open the XML file for each protein and read them.
    result_handle = open(xmlfile)
    blast_record = NCBIXML.read(result_handle)

    # Create a new text file, to store GenIndex numbers needed to download information from GenBank.
    save_file = open(xmlfile + '.txt', 'w')

    rec = 0

    # Read alignment title, split after each >, b/c hits sometimes have multiple header, and write each header in a
    # new line of the text file. Close everything in the end.
    for alignment in blast_record.alignments:
        for i in alignment.title.split(' >', alignment.title.count('>')):
            save_file.write(i + '\n')

        rec += 1

    save_file.close()
    result_handle.close()
    print xmlfile, rec

AT1G01060_BLAST.xml 1474
AT2G25930_BLAST.xml 263
AT2G40080_BLAST.xml 327
AT2G46790_BLAST.xml 10000
AT2G46830_BLAST.xml 1441
AT3G09600_BLAST.xml 1488
AT3G46640_BLAST.xml 5005
AT5G02810_BLAST.xml 10000
AT5G08330_BLAST.xml 877
AT5G24470_BLAST.xml 10000
AT5G61380_BLAST.xml 5173


In [20]:
protein = 'CHE'
# Create an empty list to store used GenIndexes.
GI_list = []

# Create a CSV file, write a header line and close is afterwards.
save_file = open('%s_BLAST.csv'%protein, 'w')
save_file.write('name,taxonomy,gene_name,gi,db,taxid,db_source,length,seq,create_date,last_update,source\n')
save_file.close()

In [21]:
# Enter your email address so NCBI knows who you are.
Entrez.email ="schmelli@msu.edu"

# Open the newly created CSV file, now with the append function, to add information from GenBank for each hit.
save_file = open('%s_BLAST.csv'%protein, 'a')

# Open text file with the headers in each line and store each line as an element in a list.
result_handle = open('%s_BLAST.txt'%protein,'r')
lines = result_handle.readlines()

# Open a text file used to store GenIndexes that can't be found.
save_file_2 = open('%s_not_found.txt'%protein, 'a')

# Loop over the list of headers and extract the GenIndex, that is usually stored at the second position.
# Send the GenIndex to GenBank to retrieve information about organism name, taxonomy and taxonomy ID,
# protein annotation, multiple protein GenIndexes, sequence, sequence length, and create/update dates.
for line in lines:
    idp = line.split('|',2)[1]
    
    # Check if GenIndex is already used to avoid duplications.
    if idp not in GI_list:
        
        # Try to send GenIndex to Genbank. If you send GenIndexes to fast Genbank will delete requests. 
        # In those cases an HTTPError will occur. Just wait a couple of seconds and try again.
        try:
            handle = Entrez.efetch(db="protein", id=idp, retmode="xml")
        except HTTPError:
            time.sleep(20)
            handle = Entrez.efetch(db="protein", id=idp, retmode="xml")
        records = Entrez.read(handle, validate=False)

        # Look for an organism name in the Genbank file, if there is nothing, write the GenIndex into a different file
        # and continue with the next GenIndex.
        try:
            save_file.write(records[0]["GBSeq_organism"].replace(',',' ')+",")
        except IndexError:
            save_file_2.write(line)
            continue
            
        # In case there is an organism name, save information about organism name, taxonomy, protein annotation, 
        # and multiple protein GenIndexes.
        save_file.write(records[0]["GBSeq_taxonomy"].replace(',',' ')+",")
        save_file.write(records[0]["GBSeq_definition"].replace(',',' ')+",")
        save_file.write(records[0]["GBSeq_other-seqids"][1].replace(',',' ')+"|,")
        save_file.write(records[0]["GBSeq_other-seqids"][0].replace(',',' ')+"|,")

        found = 0
        # Find the taxonomy ID. If there is no, write an empty field in the CSV file. Do the same for a possible
        # genomic source of the protein sequence.
        for i in range(4):
            try:
                if records[0]["GBSeq_feature-table"][0]["GBFeature_quals"][i]['GBQualifier_name'] == 'db_xref':
                    save_file.write(records[0]["GBSeq_feature-table"][0]["GBFeature_quals"][i]['GBQualifier_value'].replace(',',' ')+",")
                    found = 1
                    break
            except IndexError:
                save_file_2.write(',')
        if found == 0:
            save_file.write(',')

        try:
            save_file.write(records[0]["GBSeq_source-db"].split(',',1)[0]+",")
        except KeyError:
            save_file.write(',')

        # If everything worked so far, save information about the sequence, length, create/update date, and source.
        save_file.write(records[0]["GBSeq_length"].replace(',',' ')+",")
        save_file.write(records[0]["GBSeq_sequence"].upper()+",")
        save_file.write(records[0]['GBSeq_create-date'].replace(',',' ')+",")
        save_file.write(records[0]['GBSeq_update-date'].replace(',',' ')+",")
        save_file.write(records[0]['GBSeq_source'].replace(',',' ')+"\n")

        # Wait again to make sure not many requests go per second to NCBI.
        GI_list.append(idp)
        time.sleep(1)
    
save_file.close()
save_file_2.close()
result_handle.close()

In [22]:
proteins = ['CCA1','CHE','ELF3','ELF4','LHY1','LUX','PRR5','PRR7','PRR9',
            'RVE8','TOC1']

for protein in proteins:
    
    # Create a FASTA file.
    save_file = open('%s_BLAST.faa'%protein, 'w')

    # Open the new CSV file, skip the header and write a FASTA file using the information from the CSV file.
    with open('%s_BLAST.csv'%protein, 'r') as csvfile:
        reader = csv.reader(csvfile)
        # Skip header
        next(reader) 
        for row in reader:
            # Write multiple GenIndexes, protein annotation, and organism.
            save_file.write('>'+row[3].replace('||','|')+row[4].replace('||','|')+' '+row[2]+' '+row[0] + '\n')
            # Write sequence
            save_file.write(row[8] + '\n')

    save_file.close()

[Back to Content](#Content)

<a id='BLAST back'></a>
-----

---
###Blast hits back against the genomes###
---

Next we will blast all the sequences from our first BLAST run back against the Arabidopsis genome to validate to correctness of our first run. Only hits that align to the original protein are considered a 'real' hit.

e.g.: If you first blasted TOC1 against the 'nr' database. You would blast every hit you got in this analysis back against the Arabidopsis genome. Now you get for every hit, just one hit in the genome. Next you would filter only these hits that match with TOC1.

I ran this analysis using the standalone local BLAST from the command line, b/c I had problems with the NCBI server. Installation information for BLAST can be found [here](http://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastDocs&DOC_TYPE=Download). Following is the command line script you need to run for reproduction of my results.

---

###Arabidopsis thaliana###

---

[Back to Content](#Content)

<a id='filter'></a>
-----

---
###Filtering of the reciprocal BLAST###
---

After finishing the reciprocal BLAST, we filtered these hits, that align to original input protein. For this the GenIndex numbers for each protein is needed. We took the numbers from the corresponding genome file. Only those hits that best align to the orginial protein were retained for further analyses. The BLAST results of the remaining hits were used to write a CSV file. A CSV file represents a file format easier to work with for further analyses and visualization using the python package __pandas__.

---

In [5]:
# Genbank identifiers (GI) for all 11 proteins
CCA1 = '30690518'
CHE = '15241596'
ELF3 = '15225220'
ELF4 = '18405258'
LHY1 = '145323696'
LUX = '334185766'
PRR5 = '18420797'
PRR7 = '18414032'
PRR9 = '18407171'
RVE8 = '30680926'
TOC1 = '15240235'

In [6]:
proteins = ['CCA1','CHE','ELF3','ELF4','LHY1','LUX','PRR5','PRR7','PRR9',
            'RVE8','TOC1']

GI = [CCA1,CHE,ELF3,ELF4,LHY1,LUX,PRR5,PRR7,PRR9,RVE8,TOC1]


for prot,gi in zip(proteins,GI):

    # Read the BLAST XML file and load it into the NCBIXML.parser.
    result_handle = open('%s_BLAST.xml'%prot)
    blast_records = NCBIXML.parse(result_handle)

    # Open a CSV file and write the header in the first line.
    csv = open('%s_hits.csv'%prot,'w')
    csv.write('query_name,query_organism,query_strain,query_gene,query_gi,query_length,e_value,bitscore,identity,database_species,database_gene,database_gi,database_gene_length\n')

    records = 0

    # Loop over every record in your XML and count them.
    for blast_record in blast_records:
        records += 1

        # Check the GI for the alignment. If the hit matches your protein of interest write an entry in the CSV file.
        for alignment in blast_record.alignments:
            if gi in alignment.title.lower():

                # Some hits contain 'MULTISPECIES' entries. Loop over each organism in the query title.
                for i in blast_record.query.split(' >', blast_record.query.count('>')):
                    # Organisms names are enclosed by square bracktes and we only want to keep these hits.
                    if '[' in i:
                        org_name = i.split('[',1)[1]
                        gen_name = i.split(' ',1)[1]

                        # Extract name
                        csv.write(org_name[0:org_name.find(']')].replace(',',' ').replace("'",'').replace('[','').replace(']','').lstrip())
                        csv.write(',')

                        # Extract species name
                        csv.write(org_name[0:org_name.find(']')].replace(',',' ').replace("'",'').replace('[','').replace(']','').lstrip().split(' ',i.split('[',1)[1][0:-1].count(' '))[0])
                        csv.write(',')

                        # Extract strain name
                        csv.write(' '.join(org_name[0:org_name.find(']')].replace(',',' ').replace('[','').replace(']','').lstrip().split(' ',i.split('[',1)[1][0:-1].count(' '))[1:]))
                        csv.write(',')

                        # Extract gene name
                        csv.write(str(gen_name[0:gen_name.find(' [')].replace(',',' ')))
                        csv.write(',')

                        # Extract GI of that gene
                        csv.write(str(i.split('|',2)[1]))
                        csv.write(',')
                        for hsp in alignment.hsps:

                            # Extract gene length
                            csv.write(str(blast_record.query_length))
                            csv.write(',')

                            # Extract e-value, bitscore, identity of that hit
                            csv.write(str(hsp.expect)+','+str(hsp.score)+','+str(((hsp.identities/float(len(hsp.match)))*100)))
                            csv.write(',')

                            # Extract SyPCC7942 or SyPCC6803 organims name
                            csv.write(alignment.title.split('[',1)[1][0:-1])
                            csv.write(',')

                            # Extract SyPCC7942 or SyPCC6803 gene name
                            csv.write(alignment.title.split('|',7)[6].split('[',1)[0])
                            csv.write(',')

                            # Extract SyPCC7942 or SyPCC6803 GI for that gene
                            csv.write(alignment.title.split('|',4)[3])
                            csv.write(',')

                            # Extract SyPCC7942 or SyPCC6803 gene length
                            csv.write(str(alignment.length))
                            csv.write('\n')

                            break
                    else:
                        continue

    csv.close() 

    print prot,'no. of records:',records

2.71828182846 no. of records: 2472
2.71828182846 no. of records: 1351
2.71828182846 no. of records: 403
2.71828182846 no. of records: 674
2.71828182846 no. of records: 2543
2.71828182846 no. of records: 7425
2.71828182846 no. of records: 24376
2.71828182846 no. of records: 27573
2.71828182846 no. of records: 21818
2.71828182846 no. of records: 2503
2.71828182846 no. of records: 10517


[Back to Content](#Content)

<a id='CSV'></a>
-----

---
###Merging and final filtering###
---

In the last step in the collection and the preprocessing of the raw data, we merged the two CSV file (The one with the GenBank information of each hit, and the one with the filtered hits of the reciprocal best hit analysis). After merging the two files only those records that had a complete set of information were retained for further analyses. Resulting in the end in a set of CSV files, where each protein is represented by one CSV file containing all information.

---

In [7]:
pd.set_option('mode.chained_assignment',None)

In [19]:
# Function that merges two CSV files using pandas
def merge_csv(path1, path2):
    
    # Read the first CSV file and remove all rows with incomplete information. Take a set of columns and rename them.
    df_1 = pd.read_csv(path1, index_col=False)
    df_1 = df_1.dropna()
    a = df_1[['name','taxonomy','taxid','gene_name','db','length','seq','db_source',
              'create_date','last_update']]
    a.columns = ['query_name','taxonomy','taxid','gene_name','db','query_length','seq',
                 'db_source','create_date','last_update']
    
    # Read second CSV file and remove all rows with incomplete information.
    df_2 = pd.read_csv(path2, index_col=False)
    df_2 = df_2.dropna()
    b = df_2[['query_name','query_organism','query_gi','query_length','e_value','bitscore',
              'identity']]
    
    # Merge the two CSV files.
    c = pd.merge(a,b, how='left')
    
    # Remove again all rows with incomplete information, just to make sure the merging did not create any NaN values.
    # Next retain all hits that have an genome source, that contains the word 'accession'. Remove all duplicates.
    # In the end reset the index, and reorder and rename columns.
    c = c.dropna()
    c = c[c.db_source.str.contains('accession')]
    c = c.drop_duplicates(subset=['query_name','seq'])
    c.index = range(1,len(c)+1)
    c = c[['query_name','query_organism','taxonomy','taxid','gene_name','db','query_gi',
           'db_source','query_length','e_value','bitscore','identity','seq','create_date',
           'last_update']]
    
    name = path1.split('_',1)[0]
    
    c.columns = ['query_name','query_organism','taxonomy','taxid','gene_name','db',name+'_gi',
                 'db_source',name+'_length','e_value','bitscore','identity','seq',
                 'create_date','last_update']
    
    return c

In [20]:
# Create the merge CSV file for every single protein
proteins = ['CCA1','CHE','ELF3','ELF4','LHY1','LUX','PRR5','PRR7','PRR9',
            'RVE8','TOC1']

for i in proteins:
    
    abc = merge_csv('%s_BLAST.csv'%i,
                      '%s_hits.csv'%i)

    abc.to_csv('%s_all.csv'%i, index=False)

###Further analyses###
---

All of the analyses performed using this set of CSV files created in the code above can be found in the following IPython notebook.

+ [Distribution of circadian clock proteins from plants](2_Plant_Clock_Heatmap.ipynb)

---